In [18]:
import pandas as pd
import numpy as np 
import networkx as nx
import pickle as pkl

#dt = 'FinCausal'
dt = 'Risk'
#dt = 'Twitter'

df = pd.read_csv(dt + '.csv')

In [19]:
df

,Cause,Effect,Label,Topic_Cause,Topic_Effect,5w1h_effect,5w1h_cause
0,new competitors [2],increasing profits for our clients [13],causal,0_for_of_business_and,0_for_of_business_and,"{'Who': 'Not mentioned', 'What': 'Increasing p...","{'Who': 'Not mentioned', 'What': 'New competit..."
1,new competitors [2],"increased speed efficiency, and lower cost [11]",causal,0_for_of_business_and,-1_in_and_to_increased,"{'Who': 'Not mentioned', 'What': 'Increased sp...","{'Who': 'Not mentioned', 'What': 'New competit..."
2,"increased speed efficiency, and lower cost [11]",increasing profits for our clients [13],causal,-1_in_and_to_increased,0_for_of_business_and,"{'Who': 'Not mentioned', 'What': 'Increasing p...","{'Who': 'Not mentioned', 'What': 'Increased sp..."
3,"increased speed efficiency, and lower cost [11]",increasing profits for our clients [13],causal,-1_in_and_to_increased,0_for_of_business_and,"{'Who': 'Not mentioned', 'What': 'Increasing p...","{'Who': 'Not mentioned', 'What': 'Increased sp..."
4,changing market driving the needs for new busi...,increased importance of bundling products [16],causal,0_for_of_business_and,0_for_of_business_and,"{'Who': 'Not mentioned', 'What': 'Increased im...","{'Who': 'Businesses', 'What': 'Changing market..."
...,...,...,...,...,...,...,...
799,increased interest rate [329],increase in possibility of FCPA violation [189],non_causal,1_government_in_increase_spending,-1_in_and_to_increased,"{'Who': 'Not mentioned', 'What': 'Increase in ...","{'Who': 'Not mentioned', 'What': 'Increased in..."
800,increasing size of IT market [301],substituting or adjacent industries prevailing...,non_causal,0_for_of_business_and,0_for_of_business_and,"{'Who': 'Not mentioned', 'What': 'Substituting...","{'Who': 'Not mentioned', 'What': 'Increasing s..."
801,local business motivated to cut cost to be mor...,decreased talent in workforce [115],non_causal,0_for_of_business_and,0_for_of_business_and,"{'Who': 'Not mentioned', 'What': 'Decreased ta...","{'Who': 'Local businesses', 'What': 'Cutting c..."
802,increased motivation for entrepreneurs [109],decrease in data privacy regulations [297],non_causal,0_for_of_business_and,-1_in_and_to_increased,"{'Who': 'Not mentioned', 'What': 'Decrease in ...","{'Who': 'Entrepreneurs', 'What': 'Increased mo..."


In [46]:
dic_when_fincausal = {'2010': ['2010'],
                     '2010, 2011, 2012': ['2010', '2011', '2012'],
                     '2010, 2011, and 2012': ['2010', '2011', '2012'],
                     '2010-2011': ['2010', '2011'],
                     '2011': ['2011'],
                     '2011 (and fourth quarter of 2010)' : ['2010', '2011'],
                     '2011 and 2010': ['2010', '2011'],
                     '2011 and 2012': ['2011', '2012'],
                     '2011, 2010': ['2010', '2011'],
                     '2011, 2012' : ['2011', '2012'],
                     '2011-2012' : ['2011', '2012'],
                     '2012': ['2012'],
                     '2012 and 2011' : ['2011', '2012'],
                     '2012, 2011': ['2011', '2012'],
                     '2012-12-31': ['2012'],
                     '2013' : ['2013'],
                     '2013, 2012' : ['2012', '2013'],
                     '2013-12-31': ['2013'],
                     '2018': ['2018'],
                     '2018, 2019, 2020': ['2018', '2019', '2020'],
                     '2018-01-01' : ['2018'],
                     '2019' : ['2019'],
                     '2019 and 2018' : ['2018', '2019'],
                     '2019 compared to 2018': ['2018', '2019'],
                     '2019, 2018' : ['2018', '2019'],
                     '2019-04-18' : ['2019'],
                     '2019-06-23' : ['2019'],
                     '2019-12-31' : ['2019'],
                     '2019-2020' : ['2019', '2020'],
                     '2020': ['2020'],
                     '2020 (compared to 2019)': ['2019', '2020'],
		             '2020 and 2019': ['2019', '2020'],
		             '2020, 2019': ['2019', '2020'],
		             '2020, 2019, and 2018': ['2018','2019', '2020'],
		             '2020-12-27': ['2020'],
		             '2020-12-31': ['2020'],
		             '2020-Q4': ['2020'],
		             '2021' : ['2021'],
		             '2021 and the years ahead': ['2021'],
                     'As of December 31, 2012 and 2011' : ['2011', '2012'],
					 'As of December 31, 2020' : ['2020'],
					 'August 2011 - December 31, 2012' : ['2011','2012'],
					 'Beginning in March' : ['Beginning in March'],
					 'Beginning in March 2020' : ['2020'],
					 'By the end of 2021' : ['2021'],
					 'December 2012' : ['2012'],
					 'December 31, 2011' : ['2011'],
					 'December 31, 2012' : ['2012'],
					 'December 31, 2012 and 2011' : ['2011', '2012'],
					 'December 31, 2013' : ['2013'],
					 'December 31, 2019' : ['2019'],
					 'December 31, 2020' : ['2020'],
                     'During 2011' : ['2011'],
					 'During 2012' : ['2012'],
					 'During 2020' : ['2020'],
					 'During the winter and spring' : ['During the winter and spring'],
					 'Current year' : ['Current year'],
					 'Current year and prior year' : ['Current year and prior year'],
					 'December 1' : ['December 1'],
                     'During the year ended December 31, 2020' : ['2020'],
					 'End of 2013' : ['2013'],
					 'End of 2019' : ['2019'],
					 'February 2020' : ['2020'],
					 'First and second quarters of the year' : ['First and second quarters of the year'],
					 'First half of 2013' : ['2013'],
					 'First half of 2020' : ['2020'],
					 'First half of the year' : ['First half of the year'],
					 'First quarter of 2012' : ['2012'],
					 'For the year ended December 31, 2020' : ['2020'],
					 'Fourth quarter of 2020' : ['2020'],
					 'January 2012' : ['2012'],
					 'January 2013' : ['2013'],
					 'January 2021' : ['2021'],
					 'July 2012, during 2012' : ['2012'],
					 'June 2012' : ['2012'],
					 'June 2019' : ['2019'],
					 'June 2019, February 2020' : ['2019', '2020'],
					 'June 2020' : ['2020'],
					 'March 2020, April 2020' : ['2020'],
                     'Primarily during the first half of the year' : ['Primarily during the first half of the year'],
					 'Prior to June 2019' : ['2019'],
					 'Second half of 2011 and first quarter of 2012' : ['2011', '2012'],
					 'Third quarter of 2013' : ['2013'],
					 'Third quarter of 2020' : ['2020'],
					 'Through December 31, 2012' : ['2012'],
					 'Through the end of 2020' : ['2020'] ,
					 'Year ended December 31, 2019' : ['2019'],
					 'Years ended December 31, 2020 and 2019' : ['2019', '2020'],
					 'Years ended December 31, 2020, 2019 and 2018' : ['2018', '2019', '2020']}

dic_who_fincausal = {
 'Airlines' : 'Airlines',
 'American Airlines' : 'Airlines',
 'Architectural Coatings – EMEA' : 'Architectural Coatings – EMEA',
 'Automotive Group' : 'Automotive',
 'Automotive OEMs' : 'Automotive',
 'Automotive group' : 'Automotive',
 'Automotive segment' : 'Automotive',
 'Banking industry' : 'Banking industry',
 'Borrowers' : 'Borrowers',
 'Businesses' : 'Businesses',
 'Businesses and consumers' : 'Businesses',
 'Businesses in a specific segment' : 'Businesses',
 'CAS Medical Systems, Inc. (CASMED)' : 'CAS Medical Systems, Inc. (CASMED)',
 'Calphalon' : 'Calphalon',
 'Casinos' : 'Casinos',
 'Clinical Labs' : 'Clinical Labs',
 'Coleman' : 'Coleman',
 'Companies' : 'Company',
 'Company' : 'Company',
 'Company (not specifically named)' : 'Company',
 'Company (not specified)' : 'Company',
 'Company employees' : 'Company',
 'Company selling enhanced surgical recovery products' : 'Company',
 'Consumer Banking and Wealth' : 'Consumers',
 'Consumers' : 'Consumers',
 'Corporate and Commercial Banking' : 'Corporations',
 'Corporate and Institutional Group' : 'Corporations',
 'Corporations' : 'Corporations',
 'Currencies' : 'Currencies',
 'Customer' : 'Customers',
 'Customers' : 'Customers',
 'Delta Air Lines' : 'Airlines',
 'Delta Air Lines and the U.S. Department of the Treasury' : 'Airlines',
 'Dx' : 'Dx',
 'Employers' : 'Employers',
 'Encore Boston Harbor' : 'Encore Boston Harbor',
 'Encore Boston Harbor developers' : 'Encore Boston Harbor',
 'Every coatings business except aerospace' : 'Every coatings business except aerospace',
 'Federal Reserve' : 'Federal Reserve',
 'Financial institutions' : 'Financial institutions',
 'General Electric' : 'General Electric',
 'Glass segment' : 'Glass segment',
 'Global population' : 'Global population',
 'Grupo Aeroméxico, LATAM' :  'Airlines',
 'Home Fragrance business unit' : 'Home Fragrance business unit',
 'Home Solutions segment' : 'Home Solutions segment',
 'Homeowners' : 'Homeowners',
 'Hunter Douglas' : 'Hunter Douglas',
 'Industrial Coatings segment' : 'Industrial segment',
 'Industrial Parts Group' : 'Industrial segment',
 'Industrial segment' : 'Industrial segment',
 'Insurance companies' : 'Insurance companies',
 'International Air Transport Association (IATA)' : 'Airlines',
 'Investors' : 'Investors',
 'Investors holding securities portfolios' : 'Investors',
 'LHFS' : 'LHFS',
 'Labcorp' : 'Labcorp',
 'Las Vegas Operations and Macau Operations' : 'Operations',
 'Macau Operations, Las Vegas Operations, Encore Boston Harbor' : 'Operations',
 'Many countries' : 'Many countries',
 'Massachusetts Gaming Commission' : 'Massachusetts Gaming Commission',
 'Medtronic' : 'Medtronic',
 'North American and European markets' : 'North American and European markets',
 'North American and international businesses' : 'North American and international businesses',
 'Optical and Specialty Materials' : 'Optical and Specialty Materials',
 'PPG' : 'PPG',
 'Paper Mate' : 'Paper Mate',
 'Project Renewal' : 'Project Renewal',
 'Project Renewal and the European Transformation Plan' : 'Project Renewal and the European Transformation Plan',
 'Refineries' : 'Refineries',
 'Refinery' : 'Refineries',
 'Restaurants, Las Vegas Operations' : 'Restaurants, Las Vegas Operations',
 'Retailers in North America and Asia Pacific markets' : 'Retailers in North America and Asia Pacific markets',
 "S&P, Moody's" : "S&P, Moody's",
 'SG&A (Selling, General & Administrative Expenses)' : 'SG&A (Selling, General & Administrative Expenses)',
 'Shareholders' : 'Shareholders',
 'Silicas business' : 'Silicas business',
 "Standard & Poor's, Fitch" : "Standard & Poor's, Fitch",
 'SunTrust' : 'SunTrust',
 'Technology businesses' : 'Technology businesses',
 'The Company' : 'The Company',
 "The Company's Chief Executive Officer" : 'The Company',
 'The Company, Dx' : 'The Company',
 'The Company, Industrial Group' : 'The Company',
 'The Company, holders of Convertible Notes' : 'The Company',
 'The International Monetary Fund (IMF)' : 'The International Monetary Fund (IMF)',
 'The World Bank' : 'The World Bank',
 'The company (speaker not identified)' : 'The Company',
 'The company (speaker)' : 'The Company',
 'The company receiving payroll support payments' : 'The Company',
 'The company reporting earnings' : 'The Company',
 'The company reporting the information' : 'The Company',
 'The refinery' : 'The refinery',
 'The speaker (likely a company representative)' : 'The speaker',
 "The speaker's company (not explicitly named)" : 'The speaker',
 'Third-party refineries' : 'The refinery',
 'Transitions' : 'Transitions',
 'Truist' : 'Truist',
 'U.S. Department of the Treasury' : 'U.S. Government',
 'U.S. dollar' : 'U.S. Government',
 'US Congress' : 'U.S. Government',
 'US corporations' : 'U.S. Government',
 'US lawmakers' : 'U.S. Government',
 'United States Trade Representative (USTR)' : 'U.S. Government',
 'Wynn Las Vegas' : 'Wynn',
 'Wynn Macau' : 'Wynn',
 "Wynn Palace, Wynn Macau, the Company's Las Vegas Operations, and Encore Boston Harbor" : 'Wynn',
 'Wynn Resorts' : 'Wynn',
 'Wynn Resorts, Limited' : 'Wynn',
 'Yankee Candle' : 'Yankee Candle',
 'Zambia and its creditors' : 'Zambia and its creditors'}

dic_where_fincausal = {
 'Asia' : 'Asia',
 'Asia Pacific' : 'Asia',
 'Asian marine new-build market' : 'Asia',
 'Boston Harbor' : 'Boston Harbor',
 'Countries in which PPG operates' : 'Countries in which PPG operates',
 'Domestic region' : 'Domestic region',
 'EMEA (Europe, Middle East, and Africa)' : 'EMEA (Europe, Middle East, and Africa)',
 'EMEA segment' : 'EMEA (Europe, Middle East, and Africa)',
 'Emerging markets' : 'Emerging markets',
 'Encore Boston Harbor, Las Vegas Operations' : 'Encore Boston Harbor, Las Vegas Operations',
 'Europe' : 'Europe',
 'Europe and the United States' : 'Europe and the United States',
 'Europe, Middle East, and Africa region' : 'EMEA (Europe, Middle East, and Africa)',
 'European markets' : 'Europe',
 'European region and Brazil' : 'European region and Brazil',
 'Former chromium manufacturing plant and associated sites in New Jersey' : 'Former chromium manufacturing plant and associated sites in New Jersey',
 'Global' : 'Global',
 'Jersey City, New Jersey' : 'Jersey City, New Jersey',
 'Las Vegas' : 'Las Vegas',
 'Las Vegas Operations' : 'Las Vegas',
 'Las Vegas Operations and Encore Boston Harbor' : 'Las Vegas',
 'Las Vegas Operations, Wynn Palace, Wynn Macau, and Encore Boston Harbor' : 'Las Vegas',
 'Las Vegas and Macau' : 'Las Vegas',
 'Las Vegas, Macau' : 'Las Vegas',
 'Las Vegas, Macau, and Boston' : 'Las Vegas',
 'Las Vegas, Macau, and Boston Harbor' : 'Las Vegas',
 'Latin America and Asia Pacific' : 'Latin America and Asia Pacific',
 'Latin America and Asia Pacific regions' : 'Latin America and Asia Pacific',
 'Latin American markets and Western Europe' : 'Latin American markets and Western Europe',
 'Macau, Las Vegas, Boston Harbor' : 'Las Vegas',
 'Massachusetts' : 'Massachusetts',
 'North America' :'North America' ,
 'North America and Asia Pacific markets' : 'North America and Asia Pacific',
 'North America and Asia Pacific region' : 'North America and Asia Pacific',
 'North America and Australasia' : 'North America and Australasia',
 'North America and Europe' : 'Europe and the United States',
 'North America and International markets' : 'North America',
 'North America and international markets' : 'North America',
 'North America, Latin America' : 'North America, Latin America',
 'North America, with offsetting trends in Europe and China' : 'North America, with offsetting trends in Europe and China',
 'Primarily in the United States' : 'United States',
 'U.S. and Europe' : 'Europe and the United States',
 'United States' : 'United States',
 'Western Europe' : 'Europe',
 'Worldwide' : 'Global',
 'Wynn Palace, Wynn Macau, and Las Vegas Operations' : 'Las Vegas'}


In [4]:
df[' Effect'] = df[' Effect'].astype(str)
df[' Cause'] = df[' Cause'].astype(str)
df

,Cause,Effect,Label
0,i ned to be front and centre,it’s al about me,not-causal
1,families truly suport girl-child,we can se that sky to is not the limit,causal
2,blinding youth with pelet guns was a #comonwea...,india would win a gold every hour,not-causal
3,they were so intolerant to an individuals thou...,they cant telecast #comonwealthgames2018 medal...,causal
4,you can't wait until then,you can watch it here:,causal
...,...,...,...
911,an injury #gc2018 #gc2018wrestling,was ruled out of the final,causal
912,it’s ben shown on chanel 7 only,if it’s on 7mate or seven7 then you gota pay,not-causal
913,new zealand double their,2-0 after a goalmouth scramble,not-causal
914,they are drawing atention to themselves and th...,we aren't harasing indigenous for crimes - we ...,not-causal


In [11]:
graph = nx.Graph()

cause_col = ' Cause'
effect_col = ' Effect'

for _, row in df.iterrows():
   cause = row[cause_col]
   effect = row[effect_col]
   graph.add_edge('event:' + cause, 'relation: ' + cause + '_' + effect)
   graph.add_edge('relation: ' + cause + '_' + effect, 'event:' + effect)
   graph.nodes['relation: ' + cause + '_' + effect]['label'] = 0 if row['Label'] == 'causal' else 1

for node in graph.nodes():
      graph.nodes[node]['text'] = node
      if 'event' in node:
            graph.nodes[node]['label'] = 2

In [110]:
graph = nx.DiGraph()

cause_col = ' Cause'
effect_col = ' Effect'

for _, row in df.iterrows(): 
   
   cause = row[cause_col]
   effect = row[effect_col]
   topic_cause = row['Topic_Cause']
   topic_effect = row['Topic_Effect']
   who_cause = eval(row['5w1h_cause'])['Who']
   where_cause = eval(row['5w1h_cause'])['Where']
   when_cause = eval(row['5w1h_cause'])['When']
   who_effect = eval(row['5w1h_effect'])['Who']
   where_effect = eval(row['5w1h_effect'])['Where']
   when_effect = eval(row['5w1h_effect'])['When']
   
   graph.add_edge('event:' + cause, 'relation: ' + cause + '_' + effect)
   graph.add_edge('relation: ' + cause + '_' + effect, 'event:' + effect)
   graph.nodes['relation: ' + cause + '_' + effect]['label'] = 0 if row['Label'] == 'causal' else 1
   
   if topic_cause != '-1_the_in_and_of': 
      graph.add_edge('event:' + cause, 'topic:' + topic_cause)
      graph.add_edge('topic:' + topic_cause, 'event:' + cause)
      
   if topic_effect != '-1_the_in_and_of': 
      graph.add_edge('event:' + effect, 'topic:' + topic_effect)
      graph.add_edge('topic:' + topic_effect,'event:' + effect)
      
   if who_cause != 'Not mentioned': 
      graph.add_edge('event:' + cause, 'who:' + dic_who_fincausal[who_cause])
      graph.add_edge('who:' + dic_who_fincausal[who_cause],'event:' + cause)
      
   if where_cause != 'Not mentioned': 
      graph.add_edge('event:' + cause, 'where:' + dic_where_fincausal[where_cause])
      graph.add_edge('where:' + dic_where_fincausal[where_cause],'event:' + cause)
      
   if when_cause != 'Not mentioned': 
      for wc in dic_when_fincausal[when_cause]:
         graph.add_edge('event:' + cause, 'when:' + wc)
         graph.add_edge('when:' + wc,'event:' + cause)

   if who_effect != 'Not mentioned': 
      graph.add_edge('event:' + effect, 'who:' + dic_who_fincausal[who_effect])
      graph.add_edge('who:' + dic_who_fincausal[who_effect],'event:' + effect)
   
   if where_effect != 'Not mentioned': 
      graph.add_edge('event:' + effect, 'where:' + dic_where_fincausal[where_effect])
      graph.add_edge('where:' + dic_where_fincausal[where_effect],'event:' + effect)
      
   if when_effect != 'Not mentioned': 
      for we in dic_when_fincausal[when_effect]:
         graph.add_edge('event:' + effect, 'when:' + we)
         graph.add_edge('when:' + we,'event:' + effect)

In [111]:
for node in graph.nodes():
      graph.nodes[node]['text'] = node
      if 'event' in node:
            graph.nodes[node]['label'] = 2
      elif 'topic' in node:
            graph.nodes[node]['label'] = 3
      elif 'who' in node:
            graph.nodes[node]['label'] = 4
      elif 'where' in node:
            graph.nodes[node]['label'] = 5
      elif 'when' in node:
            graph.nodes[node]['label'] = 6

In [12]:
print(len(graph.nodes))
print(len(graph.edges))

2678
1812


In [13]:
from plotly import graph_objs as go

def show_graph(G):
  ### ARESTAS
  edge_x = []
  edge_y = []

  # adicionando as coordenadas
  for edge in G.edges():
      x0, y0 = G.nodes[edge[0]]['pos']
      x1, y1 = G.nodes[edge[1]]['pos']
      edge_x.append(x0)
      edge_x.append(x1)
      edge_x.append(None)
      edge_y.append(y0)
      edge_y.append(y1)
      edge_y.append(None)

  # definindo cor e estilo das arestas
  edge_trace = go.Scatter(
      x=edge_x, y=edge_y,
      line=dict(width=2, color='#888'),
      hoverinfo='none',
      mode='lines')

  ### VÉRTICES
  node_x = []
  node_y = []

  # adicionando as coordenadas
  text = []
  for node in G.nodes():
      x, y = G.nodes[node]['pos']
      node_x.append(x)
      node_y.append(y)
      text.append(G.nodes[node]['text']) 

  # definindo cor e estilo dos vértices
  node_trace = go.Scatter(
      x=node_x, y=node_y,
      mode='markers',
      hoverinfo='text',
      hovertext= text,
      marker=dict(
          size=10,
          line_width=2))
  
  node_labels = []
  for node in G.nodes():
    node_labels.append(G.nodes[node]['label'])

  node_trace.marker.color = node_labels
  
  # visualizando!
  fig = go.Figure(data=[edge_trace, node_trace],
              layout=go.Layout(
                  showlegend=False,
                  hovermode='closest',
                  margin=dict(b=20,l=5,r=5,t=40),
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  fig.show()

In [14]:
pos = nx.spring_layout(graph,seed=81) # obtém coordenadas dos vértices para visualização
for node in graph.nodes():
  graph.nodes[node]['pos'] = pos[node]

In [15]:
show_graph(graph)